In [56]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="white")
import os
import plotly.express as px
from plotly.subplots import make_subplots
from plotly import graph_objects as go

# Analysis of Simulation Benchmarking Experiments

The benchmarking experiments were run on  a Dell XPS-15. The metrics reported in the paper come from these experiments.

In [78]:
cols={0:"time_string",1:"time_taken_lec",2:"time_taken_safety_controller",
      3:"total_time",4:"moet",5:"mean_et",6:"avg_iterations",7:"obstacle_presence"}
algorithms = ["e2e_image_rtreach","e2e_rtreach","sac_rtreach","ars_rtreach"]
speeds = ["0.5","1.0","1.5"]
runtimes = ['10','25']
file_str = "benchmark_experiments_{}_track_porto_{}_{}.csv"
dfs = []
count = 1
for alg in algorithms:
    for speed in speeds:
        for rt in runtimes:
            file_path = file_str.format(alg,speed,rt)
            df = pd.read_csv(file_path,header=None).rename(columns=cols)
            df["ml_usage"] = df["time_taken_lec"]/df["total_time"]
            df=df.drop(columns=["time_taken_lec","total_time","time_taken_safety_controller"])
            df["sc_usage"] = 1 - df["ml_usage"]
            df["ml_usage"]=df["ml_usage"]*100
            df["sc_usage"]=df["sc_usage"]*100
            df["ml_controller"] = alg.replace("_rtreach","")
            df["speed"] = speed
            df['runtime'] = rt
            dfs.append(df)
            

In [79]:
def replace_e2e(row):
    if(row=="e2e_image"):
        return "IL (Camera)"
    elif(row=="e2e"):
        return "IL (Lidar)"
    else:
        return row.upper()
def replace_obs(row):
    if(row==0):
        return "No" 
    else: 
        return "Yes"

In [80]:
all_dfs = pd.concat(dfs)
all_dfs["ml_controller"]=all_dfs["ml_controller"].apply(replace_e2e)
all_dfs["obstacle_presence"] = all_dfs["obstacle_presence"].apply(replace_obs)
all_dfs['moet'] = all_dfs['moet'] *1000
all_dfs['mean_et'] = all_dfs['mean_et'] * 1000
all_dfs

,time_string,moet,mean_et,avg_iterations,obstacle_presence,ml_usage,sc_usage,ml_controller,speed,runtime
0,01/04/2021/17:06:07,8.478,6.64321,3.96654,No,94.933493,5.066507,IL (Camera),0.5,10
1,01/04/2021/17:07:36,8.315,6.60067,3.97587,No,88.899489,11.100511,IL (Camera),0.5,10
2,01/04/2021/17:09:06,8.507,6.43541,3.98251,No,94.946925,5.053075,IL (Camera),0.5,10
3,01/04/2021/17:10:21,8.885,6.63455,3.96435,No,94.508173,5.491827,IL (Camera),0.5,10
4,01/04/2021/17:11:51,8.353,6.52825,3.98548,No,93.997684,6.002316,IL (Camera),0.5,10
...,...,...,...,...,...,...,...,...,...,...
53,02/04/2021/21:54:04,28.240,13.85030,5.73049,Yes,23.048983,76.951017,ARS,1.5,25
54,02/04/2021/21:55:08,26.993,14.07550,5.16099,Yes,22.290988,77.709012,ARS,1.5,25
55,02/04/2021/21:56:22,28.112,14.88000,5.16214,Yes,35.960203,64.039797,ARS,1.5,25
56,02/04/2021/21:57:35,26.839,14.03640,5.09000,Yes,20.000000,80.000000,ARS,1.5,25


In [81]:
all_dfs[all_dfs['moet']>50]

,time_string,moet,mean_et,avg_iterations,obstacle_presence,ml_usage,sc_usage,ml_controller,speed,runtime
52,02/04/2021/03:35:57,116.187,14.99220,5.05786,Yes,35.931834,64.068166,IL (Lidar),1.0,25
37,02/04/2021/05:51:45,54.504,14.26180,6.10178,Yes,41.998723,58.001277,IL (Lidar),1.5,25
27,02/04/2021/10:47:49,53.933,14.33020,5.40364,No,20.909153,79.090847,SAC,1.0,25
52,02/04/2021/11:21:17,57.716,14.67240,5.21727,Yes,17.999979,82.000021,SAC,1.0,25
26,02/04/2021/14:45:39,109.141,6.35891,4.96007,No,8.166944,91.833056,SAC,1.5,10
37,02/04/2021/15:00:14,104.336,6.55656,4.44424,Yes,8.431518,91.568482,SAC,1.5,10
22,02/04/2021/13:21:26,51.835,13.59460,6.08514,No,9.420259,90.579741,SAC,1.5,25
25,02/04/2021/17:21:04,52.320,5.93489,4.83547,No,66.727559,33.272441,ARS,0.5,10
0,02/04/2021/15:29:55,54.763,13.62810,5.84139,No,60.711193,39.288807,ARS,0.5,25
18,02/04/2021/15:53:31,59.209,13.63250,5.70924,No,67.028912,32.971088,ARS,0.5,25


### Remove Outlier Measurements

In [82]:
all_dfs = all_dfs[all_dfs['moet']<50]

In [83]:
res = round(all_dfs.groupby(["ml_controller","speed","obstacle_presence","runtime"]).agg(["mean","std"]),3)
res

moet        mean_et         \
                                                 mean    std    mean    std   
ml_controller speed obstacle_presence runtime                                 
ARS           0.5   No                10       14.074  7.146   6.175  0.111   
                                      25       27.387  0.926  13.525  0.195   
                    Yes               10        9.785  1.780   6.166  0.148   
                                      25       28.991  5.063  13.827  0.492   
              1.0   No                10       15.760  4.752   6.110  0.142   
                                      25       29.271  2.451  13.767  0.213   
                    Yes               10       13.877  8.899   5.989  0.593   
                                      25       28.343  2.859  14.084  0.703   
              1.5   No                10       10.042  3.805   6.341  0.164   
                                      25       27.111  1.189  14.082  0.200   
                    Yes               10       11.037  3.684   6.269  0.232   
                                      25       27.192  0.932  14.187  0.367   
IL (Camera)   0.5   No                10        8.556  0.251   6.594  0.122   
                                      25       27.218  0.832  14.364  0.694   
                    Yes               10        8.880  0.729   6.422  0.196   
                                      25       27.528  0.973  14.444  0.293   
              1.0   No                10        8.770  0.375   6.615  0.139   
                                      25       27.392  0.672  14.208  0.232   
                    Yes               10        9.308  1.326   6.356  0.618   
                                      25       27.734  1.983  14.444  0.937   
              1.5   No                10        8.922  0.476   6.603  0.100   
                                      25       27.236  0.598  14.376  0.317   
                    Yes               10        9.570  2.078   6.113  1.109   
                                      25       27.400  1.009  14.631  0.834   
IL (Lidar)    0.5   No                10        9.031  0.310   6.620  0.077   
                                      25       28.267  0.958  14.492  0.103   
                    Yes               10        9.312  0.489   6.422  0.233   
                                      25       28.443  1.998  14.694  0.214   
              1.0   No                10        9.133  0.523   6.631  0.051   
                                      25       27.851  0.633  14.574  0.213   
                    Yes               10       10.034  2.096   6.331  0.273   
                                      25       28.538  2.282  14.516  2.501   
              1.5   No                10        9.703  1.263   6.671  0.540   
                                      25       27.876  0.728  15.080  1.264   
                    Yes               10       11.424  5.202   6.346  0.643   
                                      25       28.789  2.132  14.489  2.373   
SAC           0.5   No                10       10.514  4.313   6.391  0.183   
                                      25       27.371  1.182  13.978  0.305   
                    Yes               10       10.237  4.683   6.309  0.158   
                                      25       27.692  2.352  14.063  0.429   
              1.0   No                10       10.383  3.937   6.437  0.233   
                                      25       27.402  2.146  13.755  0.507   
                    Yes               10       11.134  3.866   6.155  0.260   
                                      25       27.644  2.614  13.881  1.064   
              1.5   No                10       11.215  5.751   6.284  0.253   
                                      25       28.066  2.239  13.433  1.096   
                    Yes               10       11.810  5.565   6.295  0.286   
                                      25       27.670  1.790  13.947  0.391   

                   

### Analysis of Hardware Experiments

In [87]:
hardware_experiments = ['benchmark_hardware_experiments0.5ms10.csv','benchmark_hardware_experiments0.5ms.csv','benchmark_hardware_experiments1.0ms.csv']
hw_dfs = []
for i in hardware_experiments:
    df = pd.read_csv(i,header=None).rename(columns=cols)
    df["ml_usage"] = df["time_taken_lec"]/df["total_time"]
    df=df.drop(columns=["time_taken_lec","time_taken_safety_controller","obstacle_presence"])
    df["sc_usage"] = 1 - df["ml_usage"]
    df["ml_usage"]=df["ml_usage"]*100
    df["sc_usage"]=df["sc_usage"]*100
 
    hw_dfs.append(df)
    
hw_dfs = pd.concat(hw_dfs)
hw_dfs['moet'] = hw_dfs['moet'] *1000
hw_dfs['mean_et'] = hw_dfs['mean_et'] * 1000
hw_dfs[hw_dfs['total_time']>5]


,time_string,total_time,moet,mean_et,avg_iterations,ml_usage,sc_usage
1,03/04/2021/12:45:08,9.9904,49.253,5.47311,4.74280,16.373819,83.626181
2,03/04/2021/12:46:39,11.7534,11.323,6.82345,4.57224,30.524019,69.475981
3,03/04/2021/12:48:11,10.0051,17.033,5.54752,4.81197,13.662032,86.337968
4,03/04/2021/12:49:39,10.0668,11.699,6.26064,4.96774,20.967636,79.032364
5,03/04/2021/12:51:33,10.9287,13.382,6.25251,4.72934,21.866096,78.133904
...,...,...,...,...,...,...,...
16,03/04/2021/14:04:57,14.1615,24.675,11.07500,5.51426,16.399251,83.600749
17,03/04/2021/14:06:34,13.6544,27.055,10.92770,5.72319,16.406946,83.593054
18,03/04/2021/14:08:12,14.3139,23.780,11.25490,5.40054,17.114693,82.885307
19,03/04/2021/14:09:44,14.9690,23.573,11.83290,5.44554,20.892845,79.107155
